In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
chrom_file = 'BL14_atleast.2_top-1_singleCpG_final_assignedref_uniq.txt'
DMR_file = 'BL14_atleast.2_top-1_singleCpG_final_record.txt'

chrom_data = pd.read_csv(chrom_file, sep='\t')#, index_col=0)
DMR_data = pd.read_csv(DMR_file, sep='\t')

df_chrom = pd.DataFrame(chrom_data)
df_DMR = pd.DataFrame(DMR_data)


In [3]:
chrom = list(df_chrom['chrom'])
start = list(df_chrom['start'])

df_DMR_new = pd.DataFrame(columns=list(df_DMR.columns))

df_chrom

,chrom,start,end,celltype
0,chr1,12168469,12168471,[Tr]
1,chr1,12168485,12168487,[Tr]
2,chr1,12168489,12168491,[Tr]
3,chr1,12168553,12168555,[Tr]
4,chr1,12168794,12168796,[Tr]
...,...,...,...,...
6866,chr8,66026577,66026579,[CD4]
6867,chr8,66026582,66026584,[CD4]
6868,chr8,66026584,66026586,[CD4]
6869,chr8,66026708,66026710,[CD4]


In [4]:
for i in range(len(chrom)):
    df_out = df_DMR[(df_DMR['chrom'] == chrom[i]) & (df_DMR['start'] == start[i])]
    if len(df_out) != 1:
        print(len(df_out))
    df_DMR_new = pd.concat([df_DMR_new, df_out])

In [5]:
df_DMR_new

,DMRchr,DMRstart,DMRend,chrom,start,end,celltype
0,chr1,12168470,12169152,chr1,12168469,12168471,[Tr]
1,chr1,12168470,12169152,chr1,12168485,12168487,[Tr]
2,chr1,12168470,12169152,chr1,12168489,12168491,[Tr]
3,chr1,12168470,12169152,chr1,12168553,12168555,[Tr]
4,chr1,12168470,12169152,chr1,12168794,12168796,[Tr]
...,...,...,...,...,...,...,...
6866,chr8,66026537,66026883,chr8,66026577,66026579,[CD4]
6867,chr8,66026537,66026883,chr8,66026582,66026584,[CD4]
6868,chr8,66026537,66026883,chr8,66026584,66026586,[CD4]
6869,chr8,66026537,66026883,chr8,66026708,66026710,[CD4]


In [6]:
def generate_df(df_DMRs, ratio):
    df_quart = pd.DataFrame(columns=df_DMRs.columns)
    cells = np.array(df_DMRs.drop_duplicates(subset=['celltype'])['celltype'])
    
    for cell in cells:
        df_cell = df_DMR_new[df_DMR_new['celltype'] == cell]
        df_cell_unique = df_cell.drop_duplicates(subset=['DMRchr', 'DMRstart', 'DMRend'])
        print(len(df_cell_unique))
        num = int(len(df_cell_unique) * ratio)
        df_samp = df_cell_unique.sample(n=num)
        DMR_chr = np.array(df_samp['DMRchr'])
        DMR_start = np.array(df_samp['DMRstart'])
        df_final = df_cell[(df_cell['DMRchr'].isin(DMR_chr)) & (df_cell['DMRstart'].isin(DMR_start))]
        df_quart = pd.concat([df_quart, df_final])
        
    return df_quart

In [7]:
df_quart = generate_df(df_DMR_new, 1/4)
display(df_quart)

df_half = generate_df(df_DMR_new, 1/2)
display(df_half)

df_3quarts = generate_df(df_DMR_new, 3/4)
display(df_3quarts)

17
5
74
26
18
73
9
23
133
73
61
170
89
21


,DMRchr,DMRstart,DMRend,chrom,start,end,celltype
102,chr2,203835818,203836888,chr2,203835817,203835819,[Tr]
103,chr2,203835818,203836888,chr2,203835825,203835827,[Tr]
104,chr2,203835818,203836888,chr2,203835855,203835857,[Tr]
105,chr2,203835818,203836888,chr2,203835876,203835878,[Tr]
106,chr2,203835818,203836888,chr2,203835888,203835890,[Tr]
...,...,...,...,...,...,...,...
6834,chr22,19884272,19884493,chr22,19884341,19884343,[CD4]
6835,chr22,19884272,19884493,chr22,19884399,19884401,[CD4]
6836,chr22,19884272,19884493,chr22,19884401,19884403,[CD4]
6837,chr22,19884272,19884493,chr22,19884419,19884421,[CD4]


17
5
74
26
18
73
9
23
133
73
61
170
89
21


,DMRchr,DMRstart,DMRend,chrom,start,end,celltype
31,chr10,17660300,17660757,chr10,17660299,17660301,[Tr]
32,chr10,17660300,17660757,chr10,17660476,17660478,[Tr]
33,chr10,17660300,17660757,chr10,17660540,17660542,[Tr]
34,chr10,17660300,17660757,chr10,17660560,17660562,[Tr]
35,chr10,17660300,17660757,chr10,17660598,17660600,[Tr]
...,...,...,...,...,...,...,...
6866,chr8,66026537,66026883,chr8,66026577,66026579,[CD4]
6867,chr8,66026537,66026883,chr8,66026582,66026584,[CD4]
6868,chr8,66026537,66026883,chr8,66026584,66026586,[CD4]
6869,chr8,66026537,66026883,chr8,66026708,66026710,[CD4]


17
5
74
26
18
73
9
23
133
73
61
170
89
21


,DMRchr,DMRstart,DMRend,chrom,start,end,celltype
0,chr1,12168470,12169152,chr1,12168469,12168471,[Tr]
1,chr1,12168470,12169152,chr1,12168485,12168487,[Tr]
2,chr1,12168470,12169152,chr1,12168489,12168491,[Tr]
3,chr1,12168470,12169152,chr1,12168553,12168555,[Tr]
4,chr1,12168470,12169152,chr1,12168794,12168796,[Tr]
...,...,...,...,...,...,...,...
6866,chr8,66026537,66026883,chr8,66026577,66026579,[CD4]
6867,chr8,66026537,66026883,chr8,66026582,66026584,[CD4]
6868,chr8,66026537,66026883,chr8,66026584,66026586,[CD4]
6869,chr8,66026537,66026883,chr8,66026708,66026710,[CD4]


In [8]:
outfol = 'ratios'
os.mkdir(outfol)

In [9]:
df_quart.to_csv(outfol + '/quarter.txt', sep='\t', index=False)

df_half.to_csv(outfol + '/half.txt', sep='\t', index=False)

df_3quarts.to_csv(outfol + '/3quarters.txt', sep='\t', index=False)